In [2]:
class TriGram():
    def __init__(self, path):
        self.unigram = {}
        self.uni_bow = {}
        self.bigram = {}
        self.bi_bow = {}
        self.trigram = {}
        self.n_gram = 0

    def _parse_arpa_from_path(self, path):
        with open(path, 'r') as arpa:
            file = arpa.readlines()


In [6]:
import re

path = './models/cs3602_1.arpa'

ngram_entries = {}
ngram_defs = {}
ngram_bows = {}

with open(path, 'r') as arpa:
    lines = [l.strip() for l in arpa.readlines()]
    line_pointer = 0
    for line_pointer in range(len(lines)):
        if lines[line_pointer] == '':
            continue
        elif lines[line_pointer] == '\\data\\' or lines[line_pointer] == '\\end\\':
            continue
        elif lines[line_pointer].startswith('ngram'):
            # match header
            ngram_header = re.findall(r'\d+', lines[line_pointer])
            ngram_header = list(map(int, ngram_header))
            order, n_entries = ngram_header
            ngram_entries[order] = n_entries
            print(order, n_entries)
        elif lines[line_pointer].endswith('-grams:'):
            order = re.findall(r'\d+', lines[line_pointer])
            order = int(order[0])
            print(f'{order}-grams')
        else:
            ngram_def = lines[line_pointer].split('\t')
            if len(ngram_def) == 3:
                log_prob, words, bow = ngram_def
            elif len(ngram_def) == 2:
                log_prob, words = ngram_def
                bow = 0.0
            words = tuple(words.split(' '))
            ngram_defs[words] = float(log_prob)
            ngram_bows[words] = float(bow)


1 12
2 70
3 14
1-grams
('0',)
('1',)
('2',)
('3',)
('4',)
('5',)
('6',)
('7',)
('8',)
('9',)
('</s>',)
('<s>',)
2-grams
('0', '2')
('0', '3')
('0', '9')
('0', '</s>')
('1', '0')
('1', '7')
('1', '9')
('1', '</s>')
('2', '3')
('2', '</s>')
('3', '0')
('3', '4')
('3', '5')
('3', '6')
('3', '7')
('3', '8')
('3', '</s>')
('4', '0')
('4', '1')
('4', '2')
('4', '3')
('4', '4')
('4', '5')
('4', '6')
('4', '7')
('4', '8')
('4', '9')
('4', '</s>')
('5', '0')
('5', '1')
('5', '2')
('5', '3')
('5', '4')
('5', '5')
('5', '6')
('5', '7')
('5', '8')
('5', '9')
('5', '</s>')
('6', '0')
('6', '1')
('6', '2')
('6', '3')
('6', '5')
('6', '6')
('6', '7')
('6', '8')
('6', '9')
('6', '</s>')
('7', '0')
('7', '1')
('7', '2')
('7', '3')
('7', '4')
('7', '5')
('7', '6')
('7', '7')
('7', '8')
('7', '9')
('7', '</s>')
('8', '0')
('8', '1')
('8', '2')
('8', '3')
('8', '4')
('8', '</s>')
('9', '0')
('9', '1')
('9', '</s>')
('<s>', '5')
3-grams
('1', '0', '3')
('3', '0', '9')
('9', '0', '3')
('5', '1', '9')
('9', 

In [3]:

print(ngram_defs)
print(ngram_bows)


{'0': -0.6210278, '1': -0.7894323, '2': -1.989005, '3': -0.7894323, '4': -1.591065, '5': -0.9963039, '6': -1.653213, '7': -1.591065, '8': -1.812913, '9': -0.7986729, '</s>': -1.113943, '<s>': -99.0, '02': -2.149219, '03': -0.1998291, '09': -0.4960066, '0</s>': -1.450249, '10': -0.3290587, '17': -1.982271, '19': -0.3388186, '1</s>': -1.283301, '23': -0.845098, '2</s>': -0.146128, '30': -0.3290587, '34': -0.9408786, '35': -0.9822713, '36': -1.028029, '37': -0.9822713, '38': -1.283301, '3</s>': -1.283301, '40': -1.176091, '41': -1.176091, '42': -1.176091, '43': -1.176091, '44': -1.176091, '45': -1.176091, '46': -1.176091, '47': -1.176091, '48': -1.176091, '49': -1.176091, '4</s>': -0.4771213, '50': -1.770852, '51': -0.1080942, '52': -1.770852, '53': -1.770852, '54': -1.770852, '55': -1.770852, '56': -1.770852, '57': -1.770852, '58': -1.770852, '59': -1.770852, '5</s>': -1.168792, '60': -1.146128, '61': -1.146128, '62': -1.146128, '63': -1.146128, '65': -1.146128, '66': -1.146128, '67': -1

In [8]:
test_string = '021033210023'

test_string = ('<s>',) + tuple(test_string) + ('</s>',)
print(test_string)

('<s>', '0', '2', '1', '0', '3', '3', '2', '1', '0', '0', '2', '3', '</s>')


In [46]:
log_probs = []

def compute_log_probability(order, words):
    if order == 1:
        words = ''.join(words)
        print(f'{words} found in lm: {ngram_defs[words]}')
        return ngram_defs[words]
    if ''.join(words) in ngram_defs:
        words = ''.join(words)
        print(f'{words} found in lm: {ngram_defs[words]}')
        return ngram_defs[words]
    elif ''.join(words[:-1]) in ngram_bows:
        return ngram_bows[''.join(words[:-1])] + compute_log_probability(order - 1, words[1:])
    else:
        return compute_log_probability(order - 1, words[1:])

first = ('<s>',)
second = ('<s>', test_string[1])

# log_probs.append(compute_log_probability(1, first))
log_probs.append(compute_log_probability(2, second))

for i in range(0, len(test_string)-2):
    log_probs.append(compute_log_probability(3, test_string[i: i+3]))


0 found in lm: -0.6210278
02 found in lm: -2.149219
1 found in lm: -0.7894323
10 found in lm: -0.3290587
103 found in lm: -0.009759837
3 found in lm: -0.7894323
2 found in lm: -1.989005
1 found in lm: -0.7894323
10 found in lm: -0.3290587
0 found in lm: -0.6210278
02 found in lm: -2.149219
23 found in lm: -0.845098
3</s> found in lm: -1.283301


In [47]:
import numpy as np

log_probs = np.array(log_probs)
exponential = - np.mean(log_probs)
ppl = 10 ** exponential
print(ppl)


58.848380854498984


In [37]:
import kenlm

m = kenlm.Model(path)
print(m.perplexity('0 2 1 0 3 3 2 1 0 0 2 3'))

58.848392301463726
